In [0]:
create or refresh streaming table circuitbox.bronze.addresses
as
select *,_metadata.file_path as input_file_name,
current_timestamp() as ingestion_time
 from cloud_files("/Volumes/circuitbox/landing/operationaldata/addresses/","csv",map("cloudFiles.inferColumnTypes" , "true"))

In [0]:
create or refresh streaming table circuitbox.silver.addresses(
  constraint validate_customer_id expect(customer_id is not null) on violation fail update,
  constraint validate_address_line_1 expect(address_line_1 is not null) on violation drop row
)
as
select customer_id,address_line_1,city,state,postcode,created_date
from stream(live.circuitbox.bronze.addresses)